## Comment Analysis

### Read Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import string
import re
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer

In [2]:
# data_raw = pd.read_csv('item_cmt.csv')
data_raw = pd.read_csv('test_shops.csv')
data_raw

,userid,shopid,itemid,rating,rating_star,comments
0,513870,4159855,35391694,1,5,NaN
1,9231308,4159855,58723194,1,5,Fast delivery Excellent service by seller
2,23942513,11433632,96151262,1,5,Good product quality Good value for money
3,6503488,4159855,197880689,1,5,NaN
4,13912163,14641014,118557053,1,5,Good product quality Good value for money
5,32196891,4159855,44881223,1,5,Good value for money Good value for money
6,10415631,4159855,258205606,1,5,Good value for money Good product quality
7,30504006,4159855,36014559,1,5,Although it's mini but it vibrated stronger th...
8,28007771,14641014,362970622,1,5,Excellent service by seller
9,10267955,4159855,33437006,1,4,NaN


In [3]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def translate(s,tran):
    global counter
    counter = counter + 1
    print counter
    try:
        return tran.translate(s,'en').text
    except:
        return ""
counter = 0
from googletrans import Translator
tran = Translator()
data_raw = data_raw.drop_duplicates()
data_raw.dropna()
data_raw = data_raw[data_raw["comments"].str.contains("Good product quality") == False]
data_raw = data_raw[data_raw["comments"].str.contains("Fast delivery") == False]
data_raw = data_raw[data_raw["comments"].str.contains("Excellent Service") == False]
# data_raw["comments"] = data_raw["comments"].apply(lambda x: x if is_ascii(x) else "")
data_raw = data_raw[data_raw["comments"]!= ""]
data_raw["comments"] = data_raw["comments"].apply(lambda x: translate(x,tran))
data_raw = data_raw[data_raw["comments"]!= ""]
# data_raw["comments"] = data_raw["comments"].apply(lambda x: x if is_ascii(x) else "")
data_raw

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


,userid,shopid,itemid,rating,rating_star,comments
5,32196891,4159855,44881223,1,5,Good value for money Good value for money
7,30504006,4159855,36014559,1,5,Although it's mini but it vibrated stronger th...
8,28007771,14641014,362970622,1,5,Excellent service by seller
10,10124621,4159855,30995977,-1,1,Take 2 months to receive the item
14,177592,4159855,30017266,1,5,Nice.
17,6369646,4159855,30974427,0,0,Mcm mne nk order
18,2338539,31964058,453870659,0,0,please who can help me
22,2955892,4159855,36263664,0,0,Can play video?
26,15347879,4159855,63008851,1,5,Good value for money
27,11130525,4159855,36014579,1,5,Excellent service by seller


In [6]:
data_raw = data_raw[data_raw["comments"].str.contains("Good value for money") == False]
data_raw

,userid,shopid,itemid,rating,rating_star,comments
7,30504006,4159855,36014559,1,5,Although it's mini but it vibrated stronger th...
8,28007771,14641014,362970622,1,5,Excellent service by seller
10,10124621,4159855,30995977,-1,1,Take 2 months to receive the item
14,177592,4159855,30017266,1,5,Nice.
17,6369646,4159855,30974427,0,0,Mcm mne nk order
18,2338539,31964058,453870659,0,0,please who can help me
22,2955892,4159855,36263664,0,0,Can play video?
27,11130525,4159855,36014579,1,5,Excellent service by seller
28,21727888,4159855,258266393,-1,1,Extra charge by the seller after the first pay...
34,7202406,4159855,30044051,0,0,Still available


In [8]:
for name, group in data_raw.groupby(by='shopid'):
    group.to_csv('%s_labelled.csv' % name, encoding='utf-8')
# data_raw.to_csv('filter_data_unlabelled.csv',header=True)

In [7]:
data = pd.read_csv('filter_data.csv')
data

,Unnamed: 0,shopid,itemid,rating_star,cmt,Sentiment
0,1,4937640,18982557,4,The thing came on time however it was broken t...,B
1,2,4149345,195378126,3,Product quality is ok Delivery time as expect...,G
2,3,4937640,40486732,1,"Delivered as a defect item, with chipped end.",B
3,6,4937640,16804582,4,"Delivery and seller, good. Product is ok? It's...",G
4,8,4149345,14482633,5,Trusted seller,G
5,10,4149345,13434358,0,Where's it made in?,N
6,11,4149345,42089797,5,"Same as photo,but wire got yellowish,box look ...",B
7,22,4937640,43305088,4,Got about 96 pieces in total but fair enough f...,G
8,23,4149345,60228536,3,Product quality is ok,G
9,25,4149345,16431297,3,Not compatible with Xbox OG,G


In [98]:
data = data[data["rating_star"]!=3]
data

,Unnamed: 0,shopid,itemid,rating_star,cmt,Sentiment
0,1,4937640,18982557,4,The thing came on time however it was broken t...,B
2,3,4937640,40486732,1,"Delivered as a defect item, with chipped end.",B
3,6,4937640,16804582,4,"Delivery and seller, good. Product is ok? It's...",G
4,8,4149345,14482633,5,Trusted seller,G
5,10,4149345,13434358,0,Where's it made in?,N
6,11,4149345,42089797,5,"Same as photo,but wire got yellowish,box look ...",B
7,22,4937640,43305088,4,Got about 96 pieces in total but fair enough f...,G
10,27,4149345,46508803,5,It works well! Item came earlier than xpected ...,G
11,28,4149345,14477372,5,Smaller than expected but still great. Not sur...,B
12,29,4937640,18981664,5,Tested working but yet to try on hair.,N


In [8]:
target = data["Sentiment"]
cmt = data["cmt"]

### Data Preprocessing

In [9]:
def tokenize(text):
    stemmer = PorterStemmer()
    
    text = re.sub(r"[^a-zA-Z]"," ",text)
    
    tokens = word_tokenize(text)
    tokens = [i for i in tokens if i not in string.punctuation]
    
    stems = stem_tokens(tokens, stemmer)
    return stems

def stem_tokens(t,s):
    stemmed=[]
    for item in t:
        stemmed.append(s.stem(item))
    return stemmed

In [10]:
vectorizer = TfidfVectorizer(stop_words='english', tokenizer = tokenize)
cmt_tfidf = vectorizer.fit_transform(cmt)

In [11]:
train, test, train_target,test_target = train_test_split(cmt_tfidf, target, test_size = 0.3, random_state = 0)

In [12]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

model = SVC()
model.fit(train, train_target)
pred = model.predict(test)
print "SVC: %f" %accuracy_score(test_target, pred)


model = RandomForestClassifier()
model.fit(train, train_target)
pred = model.predict(test)
print "Random Forest: %f" %accuracy_score(test_target, pred)


model = LogisticRegression()
model.fit(train, train_target)
pred = model.predict(test)
print "Logistic Regression: %f" %accuracy_score(test_target, pred)

model = AdaBoostClassifier()
model.fit(train, train_target)
pred = model.predict(test)
print "Adaboost: %f" %accuracy_score(test_target, pred)

model = ExtraTreesClassifier()
model.fit(train, train_target)
pred = model.predict(test)
print "Extra tree: %f" %accuracy_score(test_target, pred)

model = KNeighborsClassifier(n_neighbors=3)
model.fit(train, train_target)
pred = model.predict(test)
print "5 Nearest Neighbors: %f" %accuracy_score(test_target, pred)

model = MLPClassifier()
model.fit(train, train_target)
pred = model.predict(test)
print "Multilayer Perceptron: %f" %accuracy_score(test_target, pred)

SVC: 0.643649
Random Forest: 0.769821
Logistic Regression: 0.768116
Adaboost: 0.733163
Extra tree: 0.763001
5 Nearest Neighbors: 0.552430
Multilayer Perceptron: 0.732310


In [13]:
model = LogisticRegression()
model.get_params().keys()

['warm_start',
 'C',
 'n_jobs',
 'verbose',
 'intercept_scaling',
 'fit_intercept',
 'max_iter',
 'penalty',
 'multi_class',
 'random_state',
 'dual',
 'tol',
 'solver',
 'class_weight']

In [14]:
# tuned_parameters = {'hidden_layer_sizes':[(100,),(50,50),(30,30,30)], 'activation':['identity','logistic','tanh','relu'],
#                     'solver':['lbfgs','sgd','adam'],'alpha':[0.01,0.0001,0.000001], 'batch_size':[50,100,200],
#                     'learning_rate':['constant','invscaling','adaptive']}
tuned_parameters = {'penalty':['l2'],'class_weight':[None,'balanced'],'solver':['newton-cg','lbfgs','liblinear','sag'], 'warm_start':[True,False]}
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(LogisticRegression(), tuned_parameters, cv=5)
clf.fit(train, train_target)
print clf.best_params_

{'penalty': 'l2', 'warm_start': True, 'solver': 'liblinear', 'class_weight': 'balanced'}


In [15]:
pred = clf.predict(test)
result = clf.predict_proba(test)
print "Multilayer Perceptron: %f" %accuracy_score(test_target, pred)
print classification_report(test_target, pred)
print result


Multilayer Perceptron: 0.764706
             precision    recall  f1-score   support

          B       0.62      0.66      0.64       204
          G       0.87      0.83      0.85       755
          N       0.58      0.62      0.60       214

avg / total       0.77      0.76      0.77      1173

[[ 0.07560158  0.91412665  0.01027177]
 [ 0.27099169  0.40744314  0.32156517]
 [ 0.16887552  0.65109059  0.18003389]
 ..., 
 [ 0.01478981  0.97380675  0.01140344]
 [ 0.94304152  0.04744778  0.0095107 ]
 [ 0.32945201  0.54403728  0.1265107 ]]


### Ouput Score for All Comments

In [51]:
result_proba = clf.predict_proba(cmt_tfidf)
result = clf.predict(cmt_tfidf)
result = pd.DataFrame(result)
result.columns = ['Predicted Sentiment']
result

,Predicted Sentiment
0,B
1,G
2,B
3,G
4,G
5,N
6,B
7,G
8,G
9,N


In [52]:
result_frame = pd.DataFrame(result_proba)
result_frame.columns = ["B", "G", "N"]
result_frame["Score"] = result_frame["G"] - result_frame["B"]
result_frame = result_frame.round({'Score':2})
result_frame


,B,G,N,Score
0,0.520393,0.277533,0.202075,-0.24
1,0.209019,0.527987,0.262993,0.32
2,0.522511,0.317198,0.160291,-0.21
3,0.058375,0.712661,0.228963,0.65
4,0.239644,0.458616,0.301740,0.22
5,0.213998,0.231154,0.554848,0.02
6,0.527655,0.310328,0.162016,-0.22
7,0.288248,0.538550,0.173202,0.25
8,0.045490,0.848255,0.106255,0.80
9,0.247683,0.265433,0.486884,0.02


In [41]:
del data[data.columns.values[0]]

In [42]:
data

,shopid,itemid,rating_star,cmt,Sentiment
0,4937640,18982557,4,The thing came on time however it was broken t...,B
1,4149345,195378126,3,Product quality is ok Delivery time as expect...,G
2,4937640,40486732,1,"Delivered as a defect item, with chipped end.",B
3,4937640,16804582,4,"Delivery and seller, good. Product is ok? It's...",G
4,4149345,14482633,5,Trusted seller,G
5,4149345,13434358,0,Where's it made in?,N
6,4149345,42089797,5,"Same as photo,but wire got yellowish,box look ...",B
7,4937640,43305088,4,Got about 96 pieces in total but fair enough f...,G
8,4149345,60228536,3,Product quality is ok,G
9,4149345,16431297,3,Not compatible with Xbox OG,G


In [56]:
data.reset_index(drop=True, inplace=True)
result.reset_index(drop=True, inplace=True)
result_frame.reset_index(drop=True, inplace=True)
concat_result = pd.concat([data,result,result_frame],axis=1)
concat_result.to_csv('Result.csv',header=True)
groupby = concat_result.groupby("itemid")

In [64]:
item_score = groupby[["Score"]].median()

In [65]:
item_shopid = groupby[["shopid"]].first()

In [66]:
item_itemid = groupby[["itemid"]].first()

In [69]:
item_score.reset_index(drop=True, inplace=True)
item_shopid.reset_index(drop=True, inplace=True)
item_itemid.reset_index(drop=True, inplace=True)
item_result = pd.concat([item_itemid,item_shopid,item_score],axis = 1)
item_result.to_csv('Item_Result.csv',header=True)